# Fourier Transform

Our goal here is to model the same thing as before, using Fourier transforms

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal as multinom
from scipy.stats import norm
from statsmodels.tsa.arima_process import ArmaProcess
import matplotlib.pyplot as plt

In [ ]:
#The first term is the mean genetic value of z
#The second term is the parameter of sensitivity to maternal environment m
x0        = (1,1) #Initial conditions. 
tau       = 1000  #Lifespan
nbsteps   = 20    #Number of selective episodes in an organism’s lifetime
spacebins = 100
w2        = 10    #Inverse selection strength (constant)

V0        = (0.5,0.01)  #Genetic variance (again for l, m)
Vr        = [0.3,0.005] #This is the variability during reproduction, in the infinitesimal model

#Autocorrelation of the environment, careful it is the parameter in the equivalent Ornstein-Uhlenbeck process !
rho       = 1
sig2_e    = 20 #Variance in the environment


Tmax      = 5000

Environment

In [ ]:
def GenerateEnvironment(r=rho, s2=sig2_e, n = nbsteps*Tmax, time=tau*Tmax):
    """This function, for a given autocorrelation r and environmental standard deviation s, generates an AR[1] sequence of length n
    Keep in mind that a discretized (in steps of size dt) O-U process with parameters (r,s2) is an AR[1] process with parameters (exp(-rho dt), sqrt(s2(1-exp(-2rho dt))))"""
    x= ArmaProcess([1,-np.exp(-r*time/n)]).generate_sample(n,scale=np.sqrt(s2*(1-np.exp(-2 * r *time/n))))
    return x

def plot_spectrum(s):
    f = np.fft.rfftfreq(len(s))
    plt.loglog(f, np.abs(np.fft.rfft(s)))

def noise_psd(N, psd = lambda f,r: 1, param=1):
    X_white = np.fft.rfft(np.random.randn(N));
    S = psd(np.fft.rfftfreq(N),param)
    S = S / np.sqrt(np.mean(S**2))
    X_shaped = X_white * S;
    return np.fft.irfft(X_shaped);

def PSDGenerator(f):
    return lambda N,param=1: noise_psd(N, f, param)

@PSDGenerator
def white_noise(f,r=1):
    return 1;

@PSDGenerator
def red_noise(f,r=2):
    return 1/np.where(f == 0, float('inf'), f**r)

@PSDGenerator
def blue_noise(f,r=1):
    return np.sqrt(f);

@PSDGenerator
def violet_noise(f,r=1):
    return f;

@PSDGenerator
def brownian_noise(f,r=1):
    return 1/np.where(f == 0, float('inf'), f)

@PSDGenerator
def pink_noise(f,r=1):
    return 1/np.where(f == 0, float('inf'), np.sqrt(f))

In [ ]:
def GenerateTrajectoryMNC(x0=x0,V0=V0,Vr=Vr,w2=w2,rho=rho,sig2_e=sig2_e,nbsteps=nbsteps,tau=tau,Tmax=Tmax,th=[]):
    """This function generates the trajectory of a population for Maternal bet-hedging, No Cue:
    x0 initial means
    V0 initial variance
    Vr reproduction variance
    w2 selection strength
    rho environmental autocorrelation
    sig2_e environmental variance
    nbsteps number of selective events in an organism’s life
    tau the longevity (or time between birth and reproduction)
    Tmax number of generations to be simulated"""
    x          = x0
    z          = x0[0]
    if len(th)==0:
        th         = GenerateEnvironment(r=rho, s2=sig2_e, n = nbsteps*(Tmax+1), time=tau*(Tmax+1))
    thvarprev  = np.var(th[0:nbsteps])
    Vgenetic   = V0
    pop        = [0] 
    listx      = [[0]*6]*(Tmax-1) #6 for: l,m,varl,varm,th,thvar
    for t in range(Tmax-1):
        #Environmental parameters
        thmean    = np.mean(th[(t+1)*nbsteps:(t+2)*nbsteps])
        thvar     = np.var(th[(t+1)*nbsteps:(t+2)*nbsteps])
        #Calculation intermediates
        omega_l   =  w2+x[1]**2+2*(Vgenetic[1]-Vr[1]) #keep in mind G*(t-1) = 2(G(t)-Vr)
        omega_m   = (w2+x[1]**2+2*(Vgenetic[1]-Vr[1]))/(1 - ((x[0]-thmean)**2+Vgenetic[0])/
                                                        (w2+x[1]**2+2*(Vgenetic[1]-Vr[1])))
        eta_t     = (Vgenetic[1]-Vr[1])/(omega_m+2*(Vgenetic[1]-Vr[1])) #See the text, keep in mind G*(t-1)=2(Gt-Vr)
        #Iteration
        deltam    = np.min([np.max([-eta_t*x[1],
                                  -10*(Vr[1]+V0[1])]), #Prevents explosions
                                   10*(Vr[1]+V0[1])])
        x         = (x[0]+Vgenetic[0]/(Vgenetic[0]+omega_l)*(thmean-x[0]),
                    x[1]+deltam)
        Vgenetic  = (1/(1/omega_l  +        1/Vgenetic[0])/2+Vr[0],
                    (eta_t*Vr[1]   +(1-eta_t)*Vgenetic[1])/2+Vr[1])
        thvarprev = thvar
        listx[t]  = list(x)+list(Vgenetic)+[thmean,thvar]
    return np.transpose(listx)